In [3]:
pip install selenium 

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Lenovo\anaconda3\envs\snakes\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install webdriver_manager

  Using cached crayons-0.4.0-py2.py3-none-any.whl (4.6 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Lenovo\anaconda3\envs\snakes\python.exe -m pip install --upgrade pip' command.


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook
import time
from datetime import datetime, date
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import re
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import sys
import random
from openpyxl import load_workbook
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
import pandas as pd
import sys
from io import StringIO
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import Alert  
from selenium.webdriver.common.alert import Alert 
from ipywidgets import Button, Layout,FileUpload, IntProgress
from selenium.webdriver.chrome.options import Options

In [5]:
tab = widgets.Tab()    
up = widgets.FileUpload(accept=".csv", multiple=True,counter = 0,description='選擇CSV')
# a upload box show then click it and upload the test file
up._counter = 0
#up.observe(show_content,'value')


button_upload = widgets.Button(
    description='確認上傳',
    button_style='warning',
    tooltip='Click to Upload',
    disabled=False,
    icon='check')

button_webcrawler = widgets.Button(
    description='爬地籍資料下載KML',
    button_style='info',
    disabled=True,
    tooltip='Click to webcrawler',
    icon='bug',
    layout=Layout(width='20%', height='28px'))

out = widgets.Output(layout={'border': '1px solid black'})
#
progress = IntProgress()
# SelectMultiple widget instantiation
eraser = widgets.SelectMultiple(
    options=['tab','"'],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False
)

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)

def alert_is_present(browser):
    try:
        alert = browser.switch_to.alert
        alert.text
        return alert
    except:
        return False

def content_parser():
    if up.value == {}:
        with out:
            print('No CSV loaded')    
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == ("application/vnd.ms-excel") or ("txt/csv"):
                try:
                    content = up_value[i]["content"]
                    content_str = str(content, 'utf-8')
                except:
                    content = up_value[i]["content"]
                    content_str = str(content, 'cp950')
            if eraser.value != {}: 
                for val in eraser.value:
                    if val == "tab":
                        content_str = content_str.replace("\t","")
                    else:
                        content_str = content_str.replace(val,"")
            if content_str != "":
                str_io = StringIO(content_str) 
                return str_io
            
def df_converter():
    content = content_parser()
    if content is not None:
        df = pd.read_csv(
             content,  
             index_col=False)
        
        return df
    else:
        return None

def upload():
    df = df_converter()
    up._counter = 0
    if {'town', 'sect','number','city'}.issubset(df.columns): 
        with out:
            out.clear_output()
            print('\n欄位符合，好棒棒!!\n')
            print('\n -----Your uploaded DF looks like:----- \n')
            df=df.dropna(axis=1,how="all")
            df=df.dropna(axis=0,how="all")
            #axis =1 代表欄,0代表列
            #全部都是Na的列刪掉
            button_webcrawler.description = '爬地籍資料下載KML'
            button_webcrawler.disabled = False
            if df is not None:
                print(df)
                x_axis.options = df.columns  # Dropdown Widget update
                y_axis.options = df.columns  # Dropdown Widget update
            else:
                print('Configuration is wrong/missing...')
                
    else:
        with out:
            out.clear_output()
            df=df.dropna(axis=1,how="all")
            df=df.dropna(axis=0,how="all")
            print('\n欄位有錯啦，說明都不看清楚\n')
            button_webcrawler.description = '不給妳爬啦'
            button_webcrawler.disabled = True
            if df is not None:
                print(df)
                x_axis.options = df.columns  # Dropdown Widget update
                y_axis.options = df.columns  # Dropdown Widget update
            else:
                print('Configuration is wrong/missing...')
   
    return(df)
################################################################以上是介面設定###################################################







#随机获取一个agent
def get_UserAgent():
    return agentPools[random.randint(0,2)]




def webcrawler():
    df = upload()
    df['sect'] = df['sect'].str.strip()
    
    #创建一个agent池
    opts = Options()
    agentPools=[
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.2; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0"
    ]
    headers={'User-Agent':get_UserAgent}
    # 禁用瀏覽器彈窗避免預設路徑載入失敗
    prefs = {
    'profile.default_content_setting_values':
        {
            'notifications': 2
        }
    }
    
    #找到Google擴充套件的檔案位置(注意路徑需使用雙斜線 "\\")
    
    ######################################################改這邊################################################################
    opts.add_extension(r'C:\Users\Lenovo\AppData\Local\Google\Chrome\User Data\Default\Extensions\gjknjjomckknofjidppipffbpoekiipm\6.4.7_0.crx')
    
   ###########################################################################################################################################################################
    #將擴充套件放入至Webdriver的開啟網頁內容
    opts.add_experimental_option('prefs', prefs)
    #隱藏『Chrome正在受到自動軟體的控制』這項資訊
    #options.add_argument("disable-infobars")
    #opts.add_argument("--incognito")  # 使用無痕模式。
    # 啟動無頭模式
    #opts.add_argument('--headless')
    opts.add_argument("user-agent={}".format(get_UserAgent))         # 使用偽造的 user-agent
    opts.add_argument("--start-maximized")
    
    browser = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=opts)
    #browser.maximize_window()
    info = pd.DataFrame(columns = ['city','town','sect','number','lalo'])
    err_info = pd.DataFrame(columns = ['city','town','sect','number','lalo'])
    wait = WebDriverWait(browser, 5)
    visible = EC.visibility_of_element_located
    clickable = EC.element_to_be_clickable
    
    
    # Initialize a progess bar
    progress.max = len(df)
    progress.description = '我是進度條'
    time.sleep(0.25)
    #進入VPN介面
    browser.get("chrome-extension://gjknjjomckknofjidppipffbpoekiipm/panel/index.html")
    wait.until(clickable((By.XPATH, '//*[@id="screenMain"]/div[3]/div[1]')))
    browser.find_element_by_xpath('//*[@id="screenMain"]/div[3]/div[1]').click()

    
    for i in tqdm_notebook(range(0,len(df))):

        browser.get("https://easymap.land.moi.gov.tw/R02/Index")
        #browser.maximize_window()
        time.sleep(2)    
        #browser.find_element_by_id('').click()  # 实例化Select
        try:
            browser.find_element_by_id('notShowSysinfo_id').click()
        except:
            pass
        try:
            wait.until(clickable((By.ID, "select_city_id")))
            Select(browser.find_element_by_id('select_city_id')).select_by_visible_text(df['city'][i])
            wait.until(clickable((By.ID, "select_town_id")))
            time.sleep(5)
            browser.find_element_by_id('select_town_id').click()
            #點開下拉式選單
            Select(browser.find_element_by_id('select_town_id')).select_by_visible_text(df['town'][i])
            #點擊鄉鎮的名字
            time.sleep(2)
            wait.until(clickable((By.ID, 'select_sect_id')))        
            browser.find_element_by_id('select_sect_id').click()
            #點開區段下拉式選單
            s1 = Select(browser.find_element_by_id('select_sect_id'))
            #設置選取id範圍
            for each_option in s1.options:
                if df['sect'][i] in each_option.text:
                    s1.select_by_visible_text(each_option.text)
            #每一個在id範圍中的值去跑，如果有跟區段名字有部分吻合的就用那個text去點擊
            wait.until(clickable((By.ID, 'landno'))) 
            time.sleep(4)
            browser.find_element_by_id('landno').click()
            browser.find_element_by_id('landno').send_keys(str(df['number'][i]))
            wait.until(clickable((By.ID, 'mapTile_id')))
            browser.find_element_by_id('mapTile_id').click()
            #點開地圖型別下拉式選單
            Select(browser.find_element_by_id('mapTile_id')).select_by_visible_text("正射影像(航照圖)")
            #選擇地圖型別

            wait.until(clickable((By.ID, 'land_button')))        
            browser.find_element_by_id('land_button').click()
            time.sleep(4)  
            try:
                source = browser.find_element_by_id('OpenLayers.Layer.Vector_42_root')
                #is_displayed代表的是有沒有出現在螢幕上，會傳回布林值true 跟 false
                show = source.is_displayed()
                time.sleep(8)
                if show == True:
                    actionChains = ActionChains(browser)
                    actionChains.context_click(source).perform()
                    browser.find_element_by_xpath('/html/body/ul/li[2]').click()
                    browser.find_element_by_id('OpenLayers.Layer.Vector_42_root').click()
                #不斷點擊縮小鍵，直到出現紅紅的圖釘
                while show !=True:
                    browser.find_element_by_id('OpenLayers.Control.PanZoomBar_43_zoomout_innerImage').click()
                    source = browser.find_element_by_id('OpenLayers.Layer.Vector_42_root')
                    show = source.is_displayed()
                    actionChains = ActionChains(browser)
                    actionChains.context_click(source).perform()
                    browser.find_element_by_xpath('/html/body/ul/li[2]').click()
                    wait.until(clickable((By.ID, 'OpenLayers.Layer.Vector_42_root')))  
                    browser.find_element_by_id('OpenLayers.Layer.Vector_42_root').click()


                soup7 = BeautifulSoup(browser.page_source, "html.parser") 
                blocks = soup7.find("span", {"id": "coordDisplayLonLat"})
                pngname = str(df['town'][i]) + str(df['sect'][i]) + str(df['number'][i])
                filename = pngname + ".png"
                time.sleep(4)
                browser.save_screenshot(filename)
                com_info2 = pd.DataFrame({"city":[df['city'][i]], 
                                "town":[df['town'][i]],
                               "sect":[df['sect'][i]],
                               "number":df['number'][i],          
                               'lalo':[blocks.text]}) 
                info = pd.concat([info, com_info2], axis = 0)
                #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
                info = info.reset_index(drop = True)
                info[['latitude','longitude']] = info.lalo.str.split(",",expand=True,)
                info['latitude'] = pd.to_numeric(info['latitude'])
                info['longitude'] = pd.to_numeric(info['longitude'])
                info.to_excel("地籍資料_經緯度.xlsx")

            except:
                err_info2 = pd.DataFrame({"city":[df['city'][i]], 
                        "town":[df['town'][i]],
                       "sect":[df['sect'][i]],
                       "number":df['number'][i],          
                       'lalo':"欄位有誤"}) 
                err_info = pd.concat([err_info, err_info2], axis = 0)
                #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
                err_info = err_info.reset_index(drop = True)
                err_info.to_excel("地籍資料_錯誤.xlsx")
        except:
            err_info2 = pd.DataFrame({"city":[df['city'][i]], 
                                    "town":[df['town'][i]],
                                   "sect":[df['sect'][i]],
                                   "number":df['number'][i],          
                                   'lalo':"欄位有誤"}) 
            err_info = pd.concat([err_info, err_info2], axis = 0)
            #用一開始建立的info來跟後面的com_info2使用 concat 合併 axis=0 為直向合併
            err_info = err_info.reset_index(drop = True)
            err_info.to_excel("地籍資料_錯誤.xlsx")
            pass

            
        # 隨機暫停 1~5 秒
        time.sleep(random.uniform(1, 5))
        
        #progress bar
        progress.value += 1
        #清除cookie
        browser.delete_all_cookies()
        if progress.value in range(0,len(df),20):
            #啟動擴充套件連上VPN 
            #連結套件的html位置 chrome-extension://gjknjjomckknofjidppipffbpoekiipm/panel/index.html
            browser.get("chrome-extension://gjknjjomckknofjidppipffbpoekiipm/panel/index.html")
            browser.find_element_by_xpath('//*[@id="screenMain"]/div[3]/div[1]').click()
            time.sleep(2)
            browser.find_element_by_xpath('//*[@id="screenMain"]/div[3]/div[1]').click()
            time.sleep(3)
            
       
    progress.description = '(Done)'
        
        
        

    #Open the file to be written.
    #######
    f = open('天下地籍.kml','w',encoding = 'UTF-8')

    #Writing the kml file.
    f.write("<?xml version='1.0' encoding='UTF-8'?>\n")
    f.write("<kml xmlns='http://earth.google.com/kml/2.1'>\n")
    f.write("<Document>\n")
    for index, row in info.iterrows():
        f.write("   <Placemark>\n")
        f.write("       <name>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number']) + "</name>\n")
        f.write("       <description>" + str(row['city'])+str(row['town'])+str(row['sect'])+str(row['number'])+ "</description>\n")
        f.write("       <Point>\n")
        f.write("           <coordinates>" + str(row['longitude']) + "," + str(row['latitude']) + ","  + "</coordinates>\n")
        #因為,是文字，所以不能直接用數字加上去，要轉換成string
        f.write("       </Point>\n")
        f.write("   </Placemark>\n")
    f.write("</Document>\n")
    f.write("</kml>\n")
    f.close()
    



        
            
def preview_clicked(b):
    preview()
def webcrawler_clicked(b):
    webcrawler()
def upload_clicked(b):
    upload()



In [6]:
# Assigning functionality to buttons
button_upload.on_click(upload_clicked)
button_webcrawler.on_click(webcrawler_clicked)

accordion = widgets.Accordion(children=[
    up])

accordion_box = widgets.VBox([
    accordion,
    progress, 
    widgets.HBox([button_upload, button_webcrawler]),
    out
])
tab.set_title(0, "Upload")


In [7]:
print("維修需求，可洽許仁耀 a0937736966@gmail.com")
accordion_box


維修需求，可洽許仁耀 a0937736966@gmail.com
